# <font color='darkred'>Tese de Mestrado</font>
## Análises Descritivas

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_excel('df_final.xlsx')
df.head()

In [ ]:
df.columns

In [ ]:
# Substituição dos valores omissos por NA
df.replace('', np.nan, inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Número de NaNs em cada coluna
nans_count = df.isna().sum()

# Filtrar para mostrar apenas as colunas com valores diferentes de 0
nans_count_non_zero = nans_count[nans_count != 0]
nans_count_non_zero

#### Trabalhou/Não Trabalhou ao Longo da Década
- Criação de colunas que indicam se um trabalhador trabalhou ou não em cada ano. Utilização das colunas cae como referência uma vez que não têm nas ao longos dos 11 anos.

In [ ]:
df_trab = df.copy()

In [ ]:
df_trab.columns

In [ ]:
def coluna_trabalhou(df_trab, ano):
    coluna_nuemp = f'caem1l_{str(ano)[-2:]}'  
    coluna_trabalhou = f'trabalhou_{ano}'
    df_trab[coluna_trabalhou] = df_trab[coluna_nuemp].apply(lambda x: 'Trabalhou' if x != 'Não Trabalhou' else 'Não Trabalhou')
    return df_trab

# Criação de colunas "trabalhou_ano"
for ano in range(2009, 2020):
    df_trab = coluna_trabalhou(df_trab, ano)

In [ ]:
df_trab.head()

- Número de trabalhadores que estavam a trabalhar face aos que não estavam a trabalhar em cada ano

In [ ]:
counts = {'Ano': [], 'Trabalhou': [], 'Não trabalhou': []}

for column in df_trab.columns:
    if column.startswith('trabalhou_'):
        year = column.split('_')[1]  
        counts['Ano'].append(year)
        counts['Trabalhou'].append(df_trab[column].value_counts().get('Trabalhou', 0))
        counts['Não trabalhou'].append(df_trab[column].value_counts().get('Não Trabalhou', 0))


tab_trab = pd.DataFrame(counts)

tab_trab

#### Troca de Emprego
- Criação de colunas que indicam se ocorreu troca de emprega do ano n face ao ano n-1
Exemplo do que esta função irá fazer:

| nuemp_09       | nuemp_10       | troca_emprego_2009_2010 |
|----------------|----------------|-------------------------|
| Trabalho F     |                | Sem Informação          |
| Não Trabalhou  |                | Trocou                  |
| Trabalho A     | Trabalho A     | Manteve                 |
| Trabalho B     | Trabalho C     | Trocou                  |
| Não Trabalhou  | Não Trabalhou  | Sem Informação          |
|                | Trabalho D     | Sem Informação          |
     |


In [ ]:
def coluna_troca_emprego(df_trab, ano_atual, ano_seguinte):
    coluna_atual = f'nuemp_{str(ano_atual)[-2:]}' 
    coluna_seguinte = f'nuemp_{str(ano_seguinte)[-2:]}' 
    coluna_troca = f'troca_emprego_{ano_atual}_{ano_seguinte}'
    
    def verifica_troca(row):
        atual = row[coluna_atual]
        seguinte = row[coluna_seguinte]
        
        # Verificar se as colunas têm NaN ou strings vazias
        atual_na = pd.isna(atual) or atual == ''
        seguinte_na = pd.isna(seguinte) or seguinte == ''
        
        # Regras de troca de emprego
        if atual == 'Não Trabalhou' and seguinte == 'Não Trabalhou':
            return 'Sem Informação'
        elif atual == 'Não Trabalhou' and seguinte_na:
            return 'Trocou'
        elif atual == 'Não Trabalhou' and not seguinte_na:
            return 'Trocou'
        elif atual_na and not seguinte_na and seguinte != 'Não Trabalhou':
            return 'Sem Informação'
        elif atual_na and seguinte_na:
            return 'Sem Informação'
        elif not atual_na and not seguinte_na and atual == seguinte and atual != 'Não Trabalhou':
            return 'Manteve'
        elif not atual_na and not seguinte_na and atual != seguinte:
            return 'Trocou'
        else:
            return 'Sem Informação'
    
    df_trab[coluna_troca] = df_trab.apply(verifica_troca, axis=1)
    return df_trab

# Criar colunas de troca de emprego para os anos de 2009 a 2019
for ano in range(2009, 2019):
    df_trab = coluna_troca_emprego(df_trab, ano, ano + 1)

df_trab.head()

Criação de uma coluna que indica quantas vezes um indivíduo trocou de trabalho ao longo da década

In [ ]:
def trocas_emprego(row):
    colunas_troca = [f'troca_emprego_{ano}_{ano + 1}' for ano in range(2009, 2019)]
    return sum(row[col] == 'Trocou' for col in colunas_troca)

df_trab['total_trocas_emprego'] = df_trab.apply(trocas_emprego, axis=1)

In [ ]:
df_trab.head(2)

- Número médio de trocas de trabalho ao longos dos 11 anos.

In [ ]:
media_trocas_emprego = round(df_trab['total_trocas_emprego'].mean(),2)
print(f"A média de trocas de emprego é: {media_trocas_emprego}")

Criação de uma coluna que indica quantas vezes um indivíduo trocou de trabalho ao longo da década

In [ ]:
def manteve_emprego(row):
    colunas_troca = [f'troca_emprego_{ano}_{ano + 1}' for ano in range(2009, 2019)]
    return sum(row[col] == 'Manteve' for col in colunas_troca)

df_trab['total_manteve_emprego'] = df_trab.apply(manteve_emprego, axis=1)

In [ ]:
df_trab.tail(2)

- Número médio de vezes que os trabalhadores permaneceram na mesma empresa ao trabalho ao longos dos 11 anos.

In [ ]:
media_manteve_emprego = round(df_trab['total_manteve_emprego'].mean(),2)
print(f"A média de trocas de emprego é: {media_manteve_emprego}")

#### Número de Empresas

- Empresas distintas em cada ano

In [ ]:
na_values = ['Não Trabalhou']

columns_of_interest = [f'nuemp_{year:02}' for year in range(9, 20)]

results = []

for col in columns_of_interest:
    unique_companies = set()
    unique_values = df_trab[col].unique()  # Pegar todos os valores da coluna
    for val in unique_values:
        if pd.notna(val) and val not in na_values:
            unique_companies.add(val)
    year = 2000 + int(col[-2:])  # Ajuste para o ano completo
    results.append({
        'Ano': year,  # Usar o ano completo
        'N_emp_distinct': len(unique_companies)
    })

results_df = pd.DataFrame(results)
tab_emp = results_df.sort_values(by='Ano')
tab_emp

- Ao longo da década

In [ ]:
na_values = ['Não Trabalhou']  

columns_of_interest = [f'nuemp_{year:02}' for year in range(9, 20)]

unique_companies = set()

for col in columns_of_interest:
    unique_values = df_trab[col].unique()  
    for val in unique_values:
        if pd.notna(val) and val not in na_values:
            unique_companies.add(val)

num_unique_companies = len(unique_companies)

print(f"Número de empresas ao longo da década: {num_unique_companies}")

In [ ]:
# Junção das tab_trab e tab_emp
tab_trab['Ano'] = tab_trab['Ano'].astype(str)
tab_emp['Ano'] = tab_emp['Ano'].astype(str)

merged_df = pd.merge(tab_trab, tab_emp, on='Ano')
merged_df

In [ ]:
merged_df.to_excel('tab_trab_emp.xlsx', index=False)

#### Trocas de Trabalho ao Longo dos Anos

In [ ]:
troca_columns = [col for col in df_trab.columns if col.startswith('troca_emprego_')]

unique_values = {col: df_trab[col].unique() for col in troca_columns}

for col, unique in unique_values.items():
    print(f"Valores únicos na coluna {col}: {unique}")

In [ ]:
counts = {'Ano': [], 'Trocou': [], 'Manteve': [], 'Sem Informação': []}

for column in df_trab.columns:
    if column.startswith('troca_emprego_'):
        match = re.match(r'troca_emprego_(\d{4})_(\d{4})', column)
        if match:
            start_year, end_year = match.groups()
            formatted_year = f"{start_year}-{end_year}"
        else:
            formatted_year = column  # Caso não corresponda ao formato esperado
        
        counts['Ano'].append(formatted_year)
        counts['Trocou'].append(df_trab[column].value_counts().get('Trocou', 0))
        counts['Manteve'].append(df_trab[column].value_counts().get('Manteve', 0))
        counts['Sem Informação'].append(df_trab[column].value_counts().get('Sem Informação', 0))

tab_trocas = pd.DataFrame(counts)
tab_trocas

In [ ]:
valor_contagem = df_trab['troca_emprego_2010_2011'].value_counts()
valor_contagem

In [ ]:
tab_trocas.to_excel('tab_trocas.xlsx', index=False)

### Habilitações Literárias por Ano

In [ ]:
habil_columns = [col for col in df_trab.columns if col.startswith('habil1_')]

unique_values = {col: df_trab[col].unique() for col in habil_columns}

for col, unique in unique_values.items():
    print(f"Valores únicos na coluna {col}: {unique}")

In [ ]:
valor_contagem = df['habil1_10'].value_counts()
valor_contagem

In [ ]:
counts = {'Ano': [], 'Inferior 1º Ciclo': [], 'Básico': [], 'Secundário': [], 'Pós Secundário não Superior Nível IV': [], 
          'Bacharelato': [], 'Licenciatura': [],
          'Mestrado': [], 'Doutoramento': [], 'Ignorada': []}

for column in df_trab.columns:
    if column.startswith('habil1_'):
        year = column.split('_')[1]
        counts['Ano'].append(int(year) + 2000)
        counts['Inferior 1º Ciclo'].append(df_trab[column].value_counts().get(1, 0))
        counts['Básico'].append(df_trab[column].value_counts().get(2, 0))
        counts['Secundário'].append(df_trab[column].value_counts().get(3, 0))
        counts['Pós Secundário não Superior Nível IV'].append(df_trab[column].value_counts().get(4, 0))
        counts['Bacharelato'].append(df_trab[column].value_counts().get(5, 0))
        counts['Licenciatura'].append(df_trab[column].value_counts().get(6, 0))
        counts['Mestrado'].append(df_trab[column].value_counts().get(7, 0))
        counts['Doutoramento'].append(df_trab[column].value_counts().get(8, 0))
        counts['Ignorada'].append(df_trab[column].value_counts().get(9, 0))

tab_habil = pd.DataFrame(counts)
tab_habil

In [ ]:
tab_habil.to_excel('tab_habil.xlsx', index=False)

### Tipo de Contrato por Ano 

In [ ]:
tipo_cont1_columns = [col for col in df_trab.columns if col.startswith('tipo_contr1_')]

unique_values = {col: df_trab[col].unique() for col in tipo_cont1_columns}

for col, unique in unique_values.items():
    print(f"Valores únicos na coluna {col}: {unique}")

In [ ]:
valor_contagem = df_trab['tipo_contr1_09'].value_counts()
valor_contagem

In [ ]:
counts = {'Ano': [], 'Sem Termo': [], 'Com Termo Certo': [], 'Com Termo Incerto': [], 'Outra Situação': []}

for column in df_trab.columns:
    if column.startswith('tipo_contr1_'):
        year_part = column.split('_')[-1]
        year = int(year_part)
        counts['Ano'].append(year + 2000)
        counts['Sem Termo'].append(df_trab[column].value_counts().get(1, 0))
        counts['Com Termo Certo'].append(df_trab[column].value_counts().get(2, 0))
        counts['Com Termo Incerto'].append(df_trab[column].value_counts().get(3, 0))
        counts['Outra Situação'].append(df_trab[column].value_counts().get(8, 0))

tab_tipo_contr = pd.DataFrame(counts)
tab_tipo_contr

In [ ]:
tab_tipo_contr.to_excel('tab_tipo_contr.xlsx', index=False)

### Cae por Ano 

In [ ]:
caem1l_columns = [col for col in df_trab.columns if col.startswith('caem1l_')]

unique_values = {col: df_trab[col].unique() for col in caem1l_columns}

for col, unique in unique_values.items():
    print(f"Valores únicos na coluna {col}: {unique}")

In [ ]:
caem1l_columns = [col for col in df_trab.columns if col.startswith('caem1l_')]

unique_values_set = set()

for col in caem1l_columns:
    unique_values_set.update(df_trab[col].unique())

unique_values_list = list(unique_values_set)

print(f"Valores únicos gerais nas colunas que começam com 'prof_3d': {unique_values_list}")

In [ ]:
valor_contagem = df_trab['caem1l_09'].value_counts()
valor_contagem

In [ ]:
counts = {
    'Ano': [], 
    'Cae_A': [], 'Cae_B': [], 'Cae_C': [], 'Cae_D': [], 'Cae_E': [], 'Cae_F': [], 'Cae_G': [], 
    'Cae_H': [], 'Cae_I': [], 'Cae_J': [], 'Cae_K': [], 'Cae_L': [], 'Cae_M': [], 'Cae_N': [], 
    'Cae_O': [], 'Cae_P': [], 'Cae_Q': [], 'Cae_R': [], 'Cae_S': [], 'Cae_U': []
}

for column in df_trab.columns:
    if column.startswith('caem1l_'):
        year_part = column.split('_')[-1]
        year = int(year_part)
        counts['Ano'].append(year + 2000)
        counts['Cae_A'].append(df_trab[column].value_counts().get(1, 0))
        counts['Cae_B'].append(df_trab[column].value_counts().get(2, 0))
        counts['Cae_C'].append(df_trab[column].value_counts().get(3, 0))
        counts['Cae_D'].append(df_trab[column].value_counts().get(4, 0))
        counts['Cae_E'].append(df_trab[column].value_counts().get(5, 0))
        counts['Cae_F'].append(df_trab[column].value_counts().get(6, 0))
        counts['Cae_G'].append(df_trab[column].value_counts().get(7, 0))
        counts['Cae_H'].append(df_trab[column].value_counts().get(8, 0))
        counts['Cae_I'].append(df_trab[column].value_counts().get(9, 0))
        counts['Cae_J'].append(df_trab[column].value_counts().get(10, 0))
        counts['Cae_K'].append(df_trab[column].value_counts().get(11, 0))
        counts['Cae_L'].append(df_trab[column].value_counts().get(12, 0))
        counts['Cae_M'].append(df_trab[column].value_counts().get(13, 0))
        counts['Cae_N'].append(df_trab[column].value_counts().get(14, 0))
        counts['Cae_O'].append(df_trab[column].value_counts().get(15, 0))
        counts['Cae_P'].append(df_trab[column].value_counts().get(16, 0))
        counts['Cae_Q'].append(df_trab[column].value_counts().get(17, 0))
        counts['Cae_R'].append(df_trab[column].value_counts().get(18, 0))
        counts['Cae_S'].append(df_trab[column].value_counts().get(19, 0))
        counts['Cae_U'].append(df_trab[column].value_counts().get(21, 0))

tab_cae = pd.DataFrame(counts)
tab_cae

In [ ]:
# Mapeamento dos nomes das colunas antigas para os novos nomes
rename_map = {
    'Cae_A': 'Agricultura, produção animal, caça, floresta e pesca',
    'Cae_B': 'Indústrias extractivas',
    'Cae_C': 'Indústrias transformadoras',
    'Cae_D': 'Electricidade, gás, vapor, água quente e fria e ar frio',
    'Cae_E': 'Captação, tratamento e distribuição de água; saneamento, gestão de resíduos e despoluição',
    'Cae_F': 'Construção',
    'Cae_G': 'Comércio por grosso e a retalho; reparação de veículos automóveis e motociclos',
    'Cae_H': 'Transportes e armazenagem',
    'Cae_I': 'Alojamento, restauração e similares',
    'Cae_J': 'Actividades de informação e de comunicação',
    'Cae_K': 'Actividades financeiras e de seguros',
    'Cae_L': 'Actividades imobiliárias',
    'Cae_M': 'Actividades de consultoria, científicas, técnicas e similares',
    'Cae_N': 'Actividades administrativas e dos serviços de apoio',
    'Cae_O': 'Administração Pública e Defesa; Segurança Social Obrigatória',
    'Cae_P': 'Educação',
    'Cae_Q': 'Actividades de saúde humana e apoio social',
    'Cae_R': 'Actividades artísticas, de espectáculos, desportivas e recreativas',
    'Cae_S': 'Outras actividades de serviços',
    'Cae_U': 'Actividades dos organismos internacionais e outras instituições extra-territoriais'
}
# Renomear as colunas do DataFrame com base no mapeamento
tab_cae.rename(columns=rename_map, inplace=True)
tab_cae

In [ ]:
tab_cae.to_excel('tab_cae.xlsx', index=False)

### Prof_3d por Ano
- Tendo em consideração que se trata de uma variável categórica com mais de 100 categorias, torna-se importantes considerar apenas o Top10 de cada ano. 

In [ ]:
prof_3d_columns = [col for col in df_trab.columns if col.startswith('prof_3d_')]

unique_values_set = set()

for col in prof_3d_columns:
    unique_values_set.update(df_trab[col].unique())

unique_values_list = list(unique_values_set)

print(f"Valores únicos gerais nas colunas que começam com 'prof_3d': {unique_values_list}")

In [ ]:
prof_3d_columns = [col for col in df_trab.columns if col.startswith('prof_3d_')]

# Percorrer cada coluna e contar a frequência dos valores, apresentando apenas as 10 categorias mais comuns
for col in prof_3d_columns:
    valor_contagem = df_trab[col].value_counts().head(10)
    print(f"Top 10 valores na coluna {col}:")
    print(valor_contagem)
    print("\n")

In [ ]:
# Lista com os valores únicos dos top 10 de cada coluna
unique_top10_values_set = set()

for col in prof_3d_columns:
    top10_values = df_trab[col].value_counts().head(10).index
    unique_top10_values_set.update(top10_values)

unique_top10_values_list = list(unique_top10_values_set)

print(f"Valores únicos dos top 10 em cada coluna que começa com 'prof_3d_': {unique_top10_values_list}")

In [ ]:
unique_top10_values_list = [411, 422, 431, 432, 513, 522, 523, 524, 711, 712, 753, 815, 912, 931, 932, 962]

counts = {value: [] for value in unique_top10_values_list}
counts['Ano'] = []

for column in prof_3d_columns:
    year_part = column.split('_')[-1]
    year = int(year_part) + 2000
    counts['Ano'].append(year)
    value_counts = df_trab[column].value_counts()
    for value in unique_top10_values_list:
        counts[value].append(value_counts.get(value, 0))


tab_prof_3d = pd.DataFrame(counts)

column_order = ['Ano'] + [value for value in unique_top10_values_list]
tab_prof_3d = tab_prof_3d[column_order]

tab_prof_3d

In [ ]:
# Mapeamento dos códigos para as descrições
rename_map_prof = {
    411: 'Empregado de escritório em geral',
    422: 'Pessoal de recepção e de informação a clientes',
    431: 'Operadores de dados, de contabilidade, estatística e serviços financeiros',
    432: 'Empregados de aprovisionamento, armazém, de serviços de apoio à produção e transportes',
    513: 'Empregados de mesa e bar',
    522: 'Vendedores em lojas',
    523: 'Operadores de caixa e venda de bilhetes',
    524: 'Outros trabalhadores relacionados com vendas',
    711: 'Trabalhadores qualificados da construção das estruturas básicas e similares',
    712: 'Trabalhadores qualificados em acabamentos da construção e similares',
    753: 'Trabalhadores da confecção de vestuário, curtidores de peles, sapateiros e similares',
    815: 'Operadores de máquinas para o fabrico de produtos têxteis, de pele com pêlo e couro',
    912: 'Trabalhadores de limpeza de veículos, janelas, roupa e de outra limpeza manual',
    931: 'Trabalhadores não qualificados da indústria extractiva e construção',
    932: 'Trabalhadores não qualificados da indústria transformadora',
    962: 'Outras profissões elementares'
}

# Criar um novo mapeamento de colunas do DataFrame
rename_map_prof_df = {value: rename_map_prof[value] for value in unique_top10_values_list}

# Renomear as colunas do DataFrame com base no mapeamento
tab_prof_3d.rename(columns=rename_map_prof_df, inplace=True)
tab_prof_3d

In [ ]:
tab_prof_3d.to_excel('tab_prof_3d.xlsx', index=False)

### Distribuição dos Salários por Sexo

In [ ]:
ano_mapping = {
    '09': '2009',
    '10': '2010',
    '11': '2011',
    '12': '2012',
    '13': '2013',
    '14': '2014',
    '15': '2015',
    '16': '2016',
    '17': '2017',
    '18': '2018',
    '19': '2019'
}

In [ ]:
rganho_cols = [col for col in df_trab.columns if col.startswith('rganho_h')]

df_long = df_trab.melt(id_vars=['sexo'], value_vars=rganho_cols, var_name='ano_rganho', value_name='rganho')

# Filtrar as linhas onde 'rganho' não é "Não Trabalhou" e não é NaN
df_long = df_long[(df_long['rganho'] != "Não Trabalhou") & (df_long['rganho'].notna())]
df_long['rganho'] = pd.to_numeric(df_long['rganho'], errors='coerce')

media_rganho_por_sexo = df_long.groupby(['sexo', 'ano_rganho'])['rganho'].mean().reset_index()
tab_sexo_rganho = media_rganho_por_sexo.pivot(index='sexo', columns='ano_rganho', values='rganho')
tab_sexo_rganho.reset_index(inplace=True)
tab_sexo_rganho.columns.name = None

colunas_anos = ['sexo'] + [ano_mapping[col.split('_')[-1]] for col in tab_sexo_rganho.columns[1:]]
tab_sexo_rganho.columns = colunas_anos
tab_sexo_rganho = tab_sexo_rganho.round(2)
tab_sexo_rganho

In [ ]:
tab_sexo_rganho = tab_sexo_rganho.melt(id_vars=['sexo'], var_name='Ano', value_name='Rganho')
tab_sexo_rganho = tab_sexo_rganho.rename(columns={'sexo': 'Sexo'})
tab_sexo_rganho = tab_sexo_rganho[['Ano', 'Sexo', 'Rganho']]

In [ ]:
tab_sexo_rganho.to_excel('tab_sexo_rganho_h.xlsx', index=False)

### Distribuição dos Salários por Tipo de Contrato e por Ano

In [ ]:
ano_mapping = {
    '09': '2009',
    '10': '2010',
    '11': '2011',
    '12': '2012',
    '13': '2013',
    '14': '2014',
    '15': '2015',
    '16': '2016',
    '17': '2017',
    '18': '2018',
    '19': '2019'
}

rganho_cols = [col for col in df_trab.columns if 'rganho_h' in col]
tipo_contr_cols = [col for col in df_trab.columns if 'tipo_contr' in col]

df_long_rganho = df_trab.melt(id_vars=[], value_vars=rganho_cols, var_name='ano_rganho', value_name='rganho')
df_long_rganho['ano'] = df_long_rganho['ano_rganho'].apply(lambda x: x.split('_')[2])

df_long_tipo_contr = df_trab.melt(id_vars=[], value_vars=tipo_contr_cols, var_name='ano_tipo_contr', value_name='tipo_contr')
df_long_tipo_contr['ano'] = df_long_tipo_contr['ano_tipo_contr'].apply(lambda x: x.split('_')[2])

df_combined = pd.concat([df_long_rganho, df_long_tipo_contr['tipo_contr']], axis=1)

df_combined = df_combined[df_combined['tipo_contr'] != "Não Trabalhou"]
df_combined['rganho'] = pd.to_numeric(df_combined['rganho'], errors='coerce')
df_combined['ano'] = df_combined['ano'].map(ano_mapping)
media_rganho_tipo_contr = df_combined.groupby(['tipo_contr', 'ano'])['rganho'].mean().unstack().reset_index()

media_rganho_tipo_contr.columns.name = None
media_rganho_tipo_contr = media_rganho_tipo_contr.rename_axis(None, axis=1).fillna(0)

media_rganho_tipo_contr = media_rganho_tipo_contr.round(2)
media_rganho_tipo_contr

In [ ]:
# Inverter a tabela para estar no formato desejado no Power BI
tab_tipo_contr_rganho = media_rganho_tipo_contr.melt(id_vars=['tipo_contr'], var_name='Ano', value_name='Rganho')
tab_tipo_contr_rganho = tab_tipo_contr_rganho.rename(columns={'tipo_contr': 'Tipo de Contrato'})
tab_tipo_contr_rganho = tab_tipo_contr_rganho[['Ano', 'Tipo de Contrato', 'Rganho']]

In [ ]:
tab_tipo_contr_rganho.to_excel('tab_tipo_contr_rganho_h.xlsx', index=False)

### Média de Horas Mensais por Ano

In [ ]:
hnormais_cols = [col for col in df_trab.columns if 'hnormais' in col]

df_long_hnormais = df_trab.melt(id_vars=[], value_vars=hnormais_cols, var_name='ano_hnormais', value_name='hnormais')

df_long_hnormais['ano'] = df_long_hnormais['ano_hnormais'].apply(lambda x: f"20{x.split('_')[1]}")
df_long_hnormais['hnormais'] = pd.to_numeric(df_long_hnormais['hnormais'], errors='coerce')

# Calcular a média de hnormais por ano
media_hnormais = df_long_hnormais.groupby('ano')['hnormais'].mean().reset_index()

media_hnormais.columns = ['Ano', 'Média de Horas Normais']

media_hnormais = media_hnormais.round(0)
media_hnormais

In [ ]:
media_hnormais.to_excel('tab_media_hnormais.xlsx', index=False)

### Média de Horas Extras Mensais por Ano

In [ ]:
hnormais_cols = [col for col in df_trab.columns if 'hextra' in col]

df_long_hnormais = df.melt(id_vars=[], value_vars=hnormais_cols, var_name='ano_hextra', value_name='hextra')
df_long_hnormais['ano'] = df_long_hnormais['ano_hextra'].apply(lambda x: f"20{x.split('_')[1]}")
df_long_hnormais['hextra'] = pd.to_numeric(df_long_hnormais['hextra'], errors='coerce')

# Calcular a média de hnormais por ano
media_hnormais = df_long_hnormais.groupby('ano')['hextra'].mean().reset_index()

media_hnormais.columns = ['Ano', 'Média de Horas Extra']

media_hextras = media_hnormais.round(0)
media_hextras

In [ ]:
media_hextras.to_excel('tab_media_hextras.xlsx', index=False)

---
Beatriz Lapa - Tese de Mestrado